In [ ]:
pip install faiss-cpu

In [ ]:
import sys
sys.path.append("..") #把上级目录加入到变量中
from langchain import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
import os
import logging
from config import OPENAI_API_KEY, FAISS_DB_PATH
from query_data import get_chain, get_citations


sk = "sk"
dash = "-"
my_key= "VgaqoS0BbjxbVN7RFWcJT3BlbkFJYy3W265nnshPERsxhFMy"
os.environ['OPENAI_API_KEY'] = sk + dash + my_key

In [ ]:
# Load from existing index
rds = FAISS.load_local("../db", OpenAIEmbeddings())

In [ ]:
query = "星空如何设置单机版纸电一体开票服务？"
results = rds.similarity_search_with_score(query)
for r in results:
    print(r[0].page_content + "\n ========================")

In [ ]:
retriever = rds.as_retriever()
retriever.search_kwargs['k'] = 3
chatbot = get_chain(retriever)
# create a chat history buffer
chat_history = []

In [ ]:
# gather user input for the first question to kick off the bot
query = "星空如何设置单机版纸电一体开票服务？"

result = chatbot({"question": query, "chat_history": chat_history})

In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(temperature=0), chain_type="stuff",
                                 retriever=retriever, return_source_documents=True)

In [ ]:
# ret = qa({"query": "黑客帝国有几部？"})
ret = qa({"query": query})
f"\n更多详情，请参考：{get_citations(ret['source_documents'])}\n"

In [ ]:
from langchain.chains import ConversationalRetrievalChain

chat_history = []
qa = ConversationalRetrievalChain.from_llm(ChatOpenAI(temperature=0), retriever,
                                           return_source_documents=True)

query = "怎么申请乐企接入单位？"
result = qa({"question": query, "chat_history": chat_history})
result['answer']

In [ ]:
chat_history = [(query, result["answer"])]
query = "使用单位呢？"
result = qa({"question": query, "chat_history": chat_history})
result['answer']